In [2]:
%matplotlib inline

import os
import numpy as np
import pandas as pd
from collections import OrderedDict
import datetime
import time
import matplotlib.pyplot as plt
from etf_tools import (
    EstimatorBucket,
    prepare_etf, ez_plot,
    evaluate_stacking
)
from etf_tools.experimental import prepare_named_etf
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.utils import shuffle as sk_shuffle

from sklearn.metrics import roc_auc_score, accuracy_score
import pickle
from modeling.delay import models as delayed_model
# from modeling.delay import prep as delayed_prep

plt.style.use('ggplot')
plt.ioff()


In [60]:
src_dir = '../data/raw/groupbycode/trainingset/'
code = '00713'
sample_file = os.path.join(src_dir, f'{code}.csv')
etf = pd.read_csv(sample_file)
display(etf.shape)
display(etf.head(2))
display(etf.tail(2))
display(etf.dtypes)


(160, 41)

,code,date,name,open,high,low,close,volume,weekday,ema12,...,obv,obv_ma12,obv_ma12_diff,trend_up,trend_down,y_1,y_2,y_3,y_4,y_5
0,713,20170927,元大台灣高息低波,29.7,29.93,29.70,29.88,682,3,29.880000,...,682,682.0,0.0,0,0,29.84,29.80,29.92,30.1,30.4
1,713,20170928,元大台灣高息低波,29.9,29.95,29.82,29.84,716,4,29.873846,...,-34,324.0,-358.0,0,0,29.80,29.92,30.10,30.4,30.7


,code,date,name,open,high,low,close,volume,weekday,ema12,...,obv,obv_ma12,obv_ma12_diff,trend_up,trend_down,y_1,y_2,y_3,y_4,y_5
158,713,20180524,元大台灣高息低波,30.97,31.0,30.97,31.0,26,4,30.863798,...,13872,13781.166667,90.833333,1,0,31.0,NaN,NaN,NaN,NaN
159,713,20180525,元大台灣高息低波,30.98,31.0,30.98,31.0,6,5,30.884752,...,13878,13799.333333,78.666667,1,0,NaN,NaN,NaN,NaN,NaN


code               int64
date               int64
name              object
open             float64
high             float64
low              float64
close            float64
volume             int64
weekday            int64
ema12            float64
ema26            float64
rsv              float64
k                float64
d                float64
upward           float64
downward         float64
rs               float64
rsi              float64
kbody            float64
kbody_top        float64
kbody_bottom     float64
up_shadow        float64
low_shadow       float64
open_gap         float64
dif              float64
macd             float64
osc              float64
price_std_20     float64
atr_20           float64
atr_std_20       float64
vr               float64
obv                int64
obv_ma12         float64
obv_ma12_diff    float64
trend_up           int64
trend_down         int64
y_1              float64
y_2              float64
y_3              float64
y_4              float64


In [66]:
up_cols = ['Mon_ud', 'Tue_ud', 'Wed_ud', 'Thu_ud', 'Fri_ud']
price_cols = ['Mon_cprice', 'Tue_cprice', 'Wed_cprice', 'Thu_cprice', 'Fri_cprice']
overall_cols = ['ETFid',
                'Mon_ud', 'Mon_cprice',
                'Tue_ud', 'Tue_cprice',
                'Wed_ud', 'Wed_cprice',
                'Thu_ud', 'Thu_cprice',
                'Fri_ud', 'Fri_cprice']


def delay_predict(df):
    code = df.code[0]
    pad_num = 2 + len(str(code))
    pred_price = df.close[-5:]
    pred_updown = (-1 + 2 * (df.close[-6:].diff() >= 0)).astype(int)[-5:]
    print([code] + list(pred_updown.values) + list(pred_price.values))
    
    return pd.Series([f'{code:0{pad_num}}'] + list(pred_updown.values) + list(pred_price.values),
                        index=['ETFid'] + up_cols + price_cols).to_frame().T

In [67]:
all_codes = open('ETF_CODE', 'r').read().strip().split()
print(len(all_codes), '\n', all_codes)

18 
 ['0050', '0051', '0052', '0053', '0054', '0055', '0056', '0057', '0058', '0059', '006201', '006203', '006204', '006208', '00690', '00692', '00701', '00713']


In [68]:
output = [delay_predict(pd.read_csv(os.path.join(src_dir, f'{c}.csv'))) for c in all_codes]

[50, 1, -1, -1, 1, 1, 82.05, 81.75, 81.35, 81.55, 81.85]
[51, 1, 1, -1, 1, 1, 33.4, 33.65, 33.56, 33.61, 33.66]
[52, 1, 1, -1, 1, 1, 52.2, 52.5, 52.3, 52.3, 52.35]
[53, 1, 1, -1, 1, 1, 35.52, 35.78, 35.54, 35.59, 35.86]
[54, 1, 1, -1, 1, 1, 24.05, 24.22, 23.85, 23.87, 24.22]
[55, 1, 1, -1, 1, 1, 17.58, 17.6, 17.55, 17.57, 17.57]
[56, 1, 1, 1, 1, -1, 26.35, 26.4, 26.4, 26.41, 26.33]
[57, 1, -1, -1, -1, 1, 50.75, 50.7, 50.35, 50.15, 50.4]
[58, 1, -1, -1, -1, 1, 47.57, 47.5, 47.3, 46.85, 47.15]
[59, 1, 1, 1, -1, 1, 42.93, 43.0, 43.0, 42.87, 42.87]
[6201, 1, 1, -1, 1, 1, 14.25, 14.26, 14.13, 14.24, 14.41]
[6203, 1, 1, -1, 1, 1, 38.01, 38.26, 38.07, 38.16, 38.25]
[6204, 1, 1, -1, 1, 1, 54.5, 54.55, 54.25, 54.4, 54.55]
[6208, 1, 1, -1, 1, 1, 47.78, 48.07, 47.76, 47.83, 47.86]
[690, 1, -1, -1, 1, 1, 22.12, 22.06, 21.89, 22.03, 22.09]
[692, 1, -1, -1, 1, 1, 21.3, 21.22, 21.15, 21.19, 21.21]
[701, 1, -1, -1, -1, -1, 21.3, 21.28, 21.27, 21.25, 21.21]
[713, 1, -1, -1, 1, 1, 31.12, 31.1, 30.96, 31

In [69]:
submission = pd.concat(output, axis=0).reset_index(drop=True)[overall_cols]

In [70]:
submission.to_csv('../submits/20180528.csv', index=False)